In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import os
#pd.set_option('display.max_rows', 200)

In [2]:
data = pd.read_excel('..\\Datos Partidos\\Central Cordoba_Situaciones.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Session Start Date   39 non-null     datetime64[ns]
 1   Event                39 non-null     object        
 2   Session Name         39 non-null     object        
 3   Session Start        39 non-null     object        
 4   Session End          39 non-null     object        
 5   Tag Description      39 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            39 non-null     object        
 8   Tag End              39 non-null     object        
 9   Tag Duration (secs)  39 non-null     float64       
 10  Categoria            39 non-null     object        
 11  Zona2                39 non-null     float64       
 12  Zona                 39 non-null     float64       
 13  Accion               39 non-null     

In [3]:
data.columns = ['Fecha','etiquetado','name','Session Start','Session End','Evento','Tag Notes','Tag Start','Tag End','Tag Duration (secs)','Categoria','Zona inicial','Zona Final','Accion','Jugada']

In [4]:
data.sample(5)

,Fecha,etiquetado,name,Session Start,Session End,Evento,Tag Notes,Tag Start,Tag End,Tag Duration (secs),Categoria,Zona inicial,Zona Final,Accion,Jugada
18,2022-07-10,Central Cordoba,Situaciones,09:47:33,11:57:38,En Contra,NaN,01:20:03,01:20:05,2000.0,ATAQUE DIRECTO,2.1,4.0,VOLVER,7.0
19,2022-07-10,Central Cordoba,Situaciones,09:47:33,11:57:38,En Contra,NaN,01:20:05,01:20:07,2000.0,ATAQUE DIRECTO,2.2,2.1,VOLVER,7.0
24,2022-07-10,Central Cordoba,Situaciones,09:47:33,11:57:38,A FAVOR,NaN,01:21:30,01:21:32,2000.0,ATAQUE DIRECTO,8.2,4.0,VOLVER,8.0
12,2022-07-10,Central Cordoba,Situaciones,09:47:33,11:57:38,A FAVOR,NaN,00:26:55,00:26:57,2000.0,ATAQUE COMBINADO,8.2,5.0,REMATE ATAJADO,4.0
7,2022-07-10,Central Cordoba,Situaciones,09:47:33,11:57:38,A FAVOR,NaN,00:26:34,00:26:36,2000.0,ATAQUE COMBINADO,8.1,5.0,VOLVER,4.0


In [5]:
'''CORREGIR MAYUSCULAS Y COSAS DE EVENTOS'''

data['Evento'] = data['Evento'].str.lower().str.strip()
data['Categoria'] = data['Categoria'].str.lower().str.strip()
data['Accion'] = data['Accion'].str.lower().str.strip()

## Estadisticas 

In [6]:
data.groupby(['Evento','Categoria'])['Jugada'].nunique().to_frame('Cantidad')

Cantidad
Evento    Categoria                 
a favor   ataque combinado         2
          ataque directo           2
          pelota parada            2
          transicion               2
en contra ataque directo           4

### Armo las coordenadas

In [7]:
data['Zona inicial'] = data['Zona inicial'].replace('2A',2.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('2B',2.2,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8A',8.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8B',8.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2A',2.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2B',2.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8A',8.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8B',8.2,regex=True)

In [8]:
data['Zona inicial'].value_counts()

8.2    12
4.0     7
2.2     7
8.1     5
2.1     3
7.0     2
1.0     1
5.0     1
9.0     1
Name: Zona inicial, dtype: int64

In [9]:
data['Zona Final'].value_counts()

5.0    7
8.1    7
4.0    7
8.2    6
2.2    3
2.1    3
9.0    2
7.0    2
1.0    1
6.0    1
Name: Zona Final, dtype: int64

In [10]:
map_zona_coord_x = {  
1:45,
2.1:70,
2.2:30,
3:45,
4:136,
5:136,
6:136,
7:227,
8.1:255,
8.2:207,
9:227
}

In [11]:
map_zona_coord_y = {  
1:135,
2.1:81,
2.2:81,
3:27,
4:135,
5:81,
6:27,
7:135,
8.1:81,
8.2:81,
9:81
}

In [12]:
data['coord.final.num'] = np.where(data['Zona Final'] == 'volver',None, data['Zona Final']).astype('float')


In [13]:
data['Zona inicial x'] = data['Zona inicial'].map(map_zona_coord_x)
data['Zona Final x'] = data['coord.final.num'].map(map_zona_coord_x)
data['Zona inicial y'] = data['Zona inicial'].map(map_zona_coord_y)
data['Zona Final y'] = data['coord.final.num'].map(map_zona_coord_y)
data.iloc[:,-5:]

,coord.final.num,Zona inicial x,Zona Final x,Zona inicial y,Zona Final y
0,5.0,255,136,81,81
1,8.1,207,255,81,81
2,9.0,207,227,81,81
3,8.2,136,207,135,81
4,4.0,45,136,135,135
5,1.0,30,45,81,135
6,2.2,30,30,81,81
7,5.0,255,136,81,81
8,8.1,136,255,135,81
9,4.0,227,136,135,135


In [14]:
# pasar en ambos formatos. Crear las dos cols originales q sean strings

data['coord.inicial'] = round(data['Zona inicial x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona inicial y'],0).astype('str').str.split('.').str[0]
data['coord.final'] = round(data['Zona Final x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona Final y'],0).astype('str').str.split('.').str[0]


In [15]:
data['coord.inicial'] = np.where(data['coord.inicial']=='nan,nan', None, data['coord.inicial'])
data['coord.final'] = np.where(data['coord.final']=='nan,nan', None, data['coord.final'])

In [16]:
data[['coord.inicial','coord.final']]

,coord.inicial,coord.final
0,"255,81","136,81"
1,"207,81","255,81"
2,"207,81","227,81"
3,"136,135","207,81"
4,"45,135","136,135"
5,"30,81","45,135"
6,"30,81","30,81"
7,"255,81","136,81"
8,"136,135","255,81"
9,"227,135","136,135"


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                39 non-null     datetime64[ns]
 1   etiquetado           39 non-null     object        
 2   name                 39 non-null     object        
 3   Session Start        39 non-null     object        
 4   Session End          39 non-null     object        
 5   Evento               39 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            39 non-null     object        
 8   Tag End              39 non-null     object        
 9   Tag Duration (secs)  39 non-null     float64       
 10  Categoria            39 non-null     object        
 11  Zona inicial         39 non-null     float64       
 12  Zona Final           39 non-null     float64       
 13  Accion               39 non-null     

In [ ]:
data['Temporada'] = 'Temp 02-2022'
data['partido'] = 'Central Cordoba'

In [ ]:
data.to_excel("../Datos generados/reserva02-22-fecha7_cco_situaciones.xlsx",index=False)